In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from tqdm.notebook import tqdm
from sklearn.metrics import mean_squared_error
from sklearn import datasets, linear_model
from sklearn.feature_extraction import DictVectorizer

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC, SVR
from sklearn.model_selection import KFold
import sklearn
import warnings
warnings.filterwarnings('ignore')

import statsmodels.api as sm


In [2]:
f = pd.read_csv('/is/prices/mfd_join_lab.csv', sep='|')
f['obs'] = 1
f.head()

,ticker,date,year,day_of_week,change_close_1,change_close_2,change_close_3,change_close_4,change_close_5,change_close_6,...,label_50_2.0,label_50_3.0,label_50_4.0,label_50_5.0,label_50_6.0,label_50_7.0,label_50_8.0,label_50_9.0,label_50_10.0,obs
0,mgnt,20140402.0,2014,2,0.000000,0.357143,-1.547619,-1.876190,-5.000000,-6.785714,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,mgnt,20140402.0,2014,2,0.000000,0.357143,-1.547619,-1.876190,-5.000000,-6.785714,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,mgnt,20140402.0,2014,2,0.000000,0.357143,-1.547619,-1.876190,-5.000000,-6.785714,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,mgnt,20140402.0,2014,2,0.000000,0.357143,-1.547619,-1.876190,-5.000000,-6.785714,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,mgnt,20140902.0,2014,1,4.430633,5.527192,8.213097,6.437292,8.324084,8.879023,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1


In [4]:
for i in f.columns:
    print(f"'{i}'", sep=' ', end=', ')

'ticker', 'date', 'year', 'day_of_week', 'change_close_1', 'change_close_2', 'change_close_3', 'change_close_4', 'change_close_5', 'change_close_6', 'change_close_7', 'change_close_8', 'change_close_9', 'change_close_10', 'change_close_15', 'change_close_20', 'change_close_25', 'change_close_30', 'change_close_40', 'change_close_50', 'text_reg_spell_stop_lem', 'text_reg_spell_stop_stem', 'label_1_0.05', 'label_1_0.1', 'label_1_0.2', 'label_1_0.3', 'label_1_0.5', 'label_1_0.8', 'label_1_1.0', 'label_1_2.0', 'label_1_3.0', 'label_1_4.0', 'label_1_5.0', 'label_1_6.0', 'label_1_7.0', 'label_1_8.0', 'label_1_9.0', 'label_1_10.0', 'label_2_0.05', 'label_2_0.1', 'label_2_0.2', 'label_2_0.3', 'label_2_0.5', 'label_2_0.8', 'label_2_1.0', 'label_2_2.0', 'label_2_3.0', 'label_2_4.0', 'label_2_5.0', 'label_2_6.0', 'label_2_7.0', 'label_2_8.0', 'label_2_9.0', 'label_2_10.0', 'label_3_0.05', 'label_3_0.1', 'label_3_0.2', 'label_3_0.3', 'label_3_0.5', 'label_3_0.8', 'label_3_1.0', 'label_3_2.0', 'lab

In [5]:
f = f[['ticker', 'date', 'year', 'day_of_week', 'change_close_1', 'change_close_2', 'change_close_3', 'change_close_4', 'change_close_5', 'change_close_6', 'change_close_7', 'change_close_8', 'change_close_9', 'change_close_10', 'change_close_15', 'change_close_20', 'change_close_25', 'change_close_30', 'change_close_40', 'change_close_50', 'text_reg_spell_stop_lem', 'text_reg_spell_stop_stem', 'label_1_0.05', 'label_1_0.1', 'label_1_0.2', 'label_1_0.3', 'label_1_0.5', 'label_1_0.8', 'label_1_1.0', 'label_1_2.0', 'label_1_3.0', 'label_1_4.0', 'label_1_5.0', 'label_1_6.0', 'label_1_7.0', 'label_1_8.0', 'label_1_9.0', 'label_1_10.0', 'obs']]
f.head()

,ticker,date,year,day_of_week,change_close_1,change_close_2,change_close_3,change_close_4,change_close_5,change_close_6,...,label_1_2.0,label_1_3.0,label_1_4.0,label_1_5.0,label_1_6.0,label_1_7.0,label_1_8.0,label_1_9.0,label_1_10.0,obs
0,mgnt,20140402.0,2014,2,0.000000,0.357143,-1.547619,-1.876190,-5.000000,-6.785714,...,0,0,0,0,0,0,0,0,0,1
1,mgnt,20140402.0,2014,2,0.000000,0.357143,-1.547619,-1.876190,-5.000000,-6.785714,...,0,0,0,0,0,0,0,0,0,1
2,mgnt,20140402.0,2014,2,0.000000,0.357143,-1.547619,-1.876190,-5.000000,-6.785714,...,0,0,0,0,0,0,0,0,0,1
3,mgnt,20140402.0,2014,2,0.000000,0.357143,-1.547619,-1.876190,-5.000000,-6.785714,...,0,0,0,0,0,0,0,0,0,1
4,mgnt,20140902.0,2014,1,4.430633,5.527192,8.213097,6.437292,8.324084,8.879023,...,1,1,1,0,0,0,0,0,0,1


In [7]:
p = pd.read_csv('/is/prices/mfd_change_price.csv', sep='|')
p.date = p.date.astype(float)
p.head()

,ticker,per,date,time,open,high,low,close,vol,change_close_1,...,change_close_7,change_close_8,change_close_9,change_close_10,change_close_15,change_close_20,change_close_25,change_close_30,change_close_40,change_close_50
0,yndx,D,20210302.0,0,4968.4,5068.6,4960.0,5027.0,683676,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,yndx,D,20210301.0,0,4858.6,5009.2,4820.2,5004.6,1090605,0.447588,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,yndx,D,20210226.0,0,4851.0,4875.8,4762.0,4813.2,1438926,3.976564,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,yndx,D,20210225.0,0,4945.0,4975.0,4821.0,4857.4,906961,-0.909952,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,yndx,D,20210224.0,0,4960.0,5032.0,4901.2,4928.2,1001828,-1.436630,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
f = f.merge(p[['ticker', 'date', 'close']], on=['ticker', 'date'])
f.shape

(115127, 40)

In [12]:
print(f.shape)
b = pd.read_csv('/is/prices/mfd_bh.csv', sep='|')
f = f.merge(b, how='inner', on='ticker')
f = f.sort_values('date', ascending=False)
print(f.shape)

(115127, 40)
(115127, 41)


In [13]:
f.head()

,ticker,date,year,day_of_week,change_close_1,change_close_2,change_close_3,change_close_4,change_close_5,change_close_6,...,label_1_4.0,label_1_5.0,label_1_6.0,label_1_7.0,label_1_8.0,label_1_9.0,label_1_10.0,obs,close,bh
43044,tgka,20210402.0,2021,4,-0.167785,-0.335570,-1.392617,-0.838926,-0.201342,1.157718,...,0,0,0,0,0,0,0,1,0.011920,-4.806202
43040,tgka,20210401.0,2021,3,1.240020,1.070155,0.900289,-0.169866,0.390691,1.036181,...,0,0,0,0,0,0,0,1,0.011774,-4.806202
43042,tgka,20210302.0,2021,1,-0.433291,-0.884636,-0.974905,-0.686044,-0.252753,2.112295,...,0,0,0,0,0,0,0,1,0.011078,-4.806202
43043,tgka,20210302.0,2021,1,-0.433291,-0.884636,-0.974905,-0.686044,-0.252753,2.112295,...,0,0,0,0,0,0,0,1,0.011078,-4.806202
16651,sber,20210301.0,2021,0,2.017691,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,1,273.580000,9.450980


In [401]:
j_neut = 0.2
j_pos = 2.0
j_neg = 2.0

label_neut = f'label_{i}_{j_neut}'
label_pos = f'label_{i}_{j_pos}'
label_neg = f'label_{i}_{j_neg}'
print('neut: ', f[label_neut][f[label_neut]==0].shape)
print('neg: ', f[label_neg][f[label_neg]==-1].shape)
print('pos: ', f[label_pos][f[label_pos]==1].shape)

neut:  (11208,)
neg:  (11627,)
pos:  (12971,)


#### Добавляем переменные за н дней

In [4]:
len(f.ticker.unique())*7*3*3*3*5

54810

#### Финальный репорт

In [ ]:
from sklearn import metrics
field = 'text_reg_spell_stop_lem'
out = pd.DataFrame()

min_dfs = [5, 10, 15, 21, 28, 36, 45, 55, 66] 
max_dfs = [0.5]
ngram_range_maxs = [1]
j_neuts = [0.2, 0.5]
j_poss = [2.0, 3.0, 4.0, 6.0] 
j_negs = [2.0, 3.0, 4.0, 6.0]
days = [1]

counter = 0
i = 1
t = 4

# split
for ticker in  f.ticker.unique(): #
    print(ticker)

    for j_neut in j_neuts:
        for j_pos in j_poss:
            for j_neg in j_negs:
                label_neut = f'label_{i}_{j_neut}'
                label_pos = f'label_{i}_{j_pos}'
                label_neg = f'label_{i}_{j_neg}'
                for day in days:
                    try:

                        ps = f[[label_neut, label_pos, label_neg, field]][(f.ticker!=ticker)] 
                        ps = ps.loc[:,~ps.columns.duplicated()]
                        if t == 0:
                            min_df = min(ps[(ps[label_pos]==1)].shape[0], \
                                         ps[(ps[label_neg]==-1)].shape[0])

                            ps1 = ps[(ps[label_pos]==1)].sample(frac=1, random_state=41)
                            ps2 = ps[(ps[label_neg]==-1)].sample(frac=1, random_state=42)

                            df = pd.concat([ps1[:min_df].rename(columns={label_pos:'sentiment'}), \
                                            ps2[:min_df].rename(columns={label_neg:'sentiment'}), \
                                           ]).sample(frac=1, random_state=44)

                        else:
                            min_df = min(ps[(ps[label_pos]==1)].shape[0], \
                                         ps[(ps[label_neut]==0)].shape[0], \
                                         ps[(ps[label_neg]==-1)].shape[0])

                            ps1 = ps[(ps[label_pos]==1)][[label_pos, field]].sample(frac=1, random_state=41)
                            ps2 = ps[(ps[label_neut]==0)][[label_neut, field]].sample(frac=1, random_state=42)
                            ps3 = ps[(ps[label_neg]==-1)][[label_neg, field]].sample(frac=1, random_state=43)

                            df = pd.concat([ps1[:min_df].rename(columns={label_pos:'sentiment'}), \
                                            ps2[:min_df].rename(columns={label_neut:'sentiment'}), \
                                            ps3[:min_df].rename(columns={label_neg:'sentiment'}) \
                                           ]).sample(frac=1, random_state=44)


                        x_train = df[field]
                        y_train = df['sentiment'] 

                    # fit
                        for ngram_range_max in ngram_range_maxs:
                            for min_df in min_dfs:
                                for max_df in max_dfs:

                                    tf_idf = TfidfVectorizer(min_df=min_df, \
                                                             max_df=max_df, \
                                                             ngram_range=(1, ngram_range_max))

                                    train_features = tf_idf.fit_transform(x_train)


                                    model = MultinomialNB(alpha=0.1)
                                    model.fit(train_features, y_train)
                                    y_train_pred = model.predict(train_features)
    # В идеале волатильность взять за эти же лаги наверное
                                    df_temp = pd.DataFrame(f[(f.ticker==ticker)]\
                                                           [['text_reg_spell_stop_lem', 'obs', 'date',\
                                                             f'change_close_{day}', 'bh', 'close']]).\
        reset_index().drop('index', axis=1)

                                    y_pred = model.predict(tf_idf.transform(df_temp.text_reg_spell_stop_lem))

                                    df_temp['sentiment'] = y_pred

                                    df_temp['neg'] = 0
                                    df_temp['neg'][df_temp.sentiment==-1] = 1

                                    df_temp['pos'] = 0
                                    df_temp['pos'][df_temp.sentiment==1] = 1

                                    df_temp['neut'] = 0
                                    df_temp['neut'][df_temp.sentiment==0] = 1

                                    df_temp = df_temp.drop_duplicates().\
                                    groupby(['date', f'change_close_{day}', 'bh', 'close']).sum().reset_index()

    # Делаем сум роллинг окно для суммирования сентиментов для периодов больше 1 дня 
    # Нужно удостовериться в сортировке должно быть по возрастанию даты

                                    df_temp[f'close_lag_1'] = df_temp.close.shift(periods=1)
                                    #df_temp[f'close_change_lag_1'] = df_temp.close.shift(periods=1)
                                    df_temp = df_temp.sort_values('date', ascending=True)

                                    df_temp['neg'] = df_temp['neg'].rolling(day).sum()
                                    df_temp['pos'] = df_temp['pos'].rolling(day).sum()
                                    df_temp['neut'] = df_temp['neut'].rolling(day).sum()

                                    df_temp.dropna(subset=['neg', 'pos', 'neut', 'close_lag_1'], inplace=True)
                    # просто сумма
                                    df_temp['sum_sentiment'] = 0
                                    df_temp['sum_sentiment'][(df_temp.pos>df_temp.neg)&(df_temp.pos>=df_temp.neut)] = 1
                                    df_temp['sum_sentiment'][(df_temp.pos<df_temp.neg)&(df_temp.neg>=df_temp.neut)] = -1

    # дивергенция c добавлением 
                                    df_temp['diver'] = ((df_temp.pos - df_temp.neg)**2 \
                                                              / (df_temp.pos + df_temp.neg)**2)**(1/2)

                                    #df_temp.reset_index(level=[f'change_close_{day}', 'date', 'bh'], inplace=True)
                                    df_temp['clear_return'] = df_temp.sum_sentiment * df_temp[f'change_close_{day}']
                                    df_temp['diver_return'] = df_temp.clear_return * df_temp.diver                

                                    res = dict()
                                    res['counter'] = [counter]
                                    res['ticker'] = [ticker]
                                    res['date'] = [day]

                                    res['min_df'] = [min_df]

                                    res['j_neut'] = [j_neut]
                                    res['j_pos'] = [j_pos] 
                                    res['j_neg'] = [j_neg] 

                                    res['train_size'] = [y_train.shape[0]]
                                    res['test_size'] = [y_pred.shape[0]]

                                    res['obs'] = \
                                    [(df_temp.obs[df_temp.clear_return.isna()==False]).sum()]
                                    res['bh'] = df_temp.bh.max()
                                    res['original_return_isna_all'] = \
                                    [(df_temp[f'change_close_{day}']\
                                      [df_temp[f'change_close_{day}'].isna()==False]).sum()]


                                    res['model_return_isna_all'] = \
                                    [(df_temp.clear_return[df_temp.clear_return.isna()==False]).sum()]

                                    res['model_return_isna_short'] = \
                                    [(df_temp.clear_return[(df_temp.clear_return.isna()==False) & \
                                                                 (df_temp.sum_sentiment!=1)]).sum()]

                                    res['model_return_isna_long'] = \
                                    [(df_temp.clear_return[(df_temp.clear_return.isna()==False) & \
                                                                 (df_temp.sum_sentiment!=-1)]).sum()]


                                    res['model_diver_return_isna_all'] = \
                                    [(df_temp.diver_return[df_temp.diver_return.isna()==False]).sum()]

                                    res['model_diver_return_isna_short'] = \
                                    [(df_temp.diver_return[(df_temp.diver_return.isna()==False) & \
                                                                 (df_temp.sum_sentiment!=1)]).sum()]

                                    res['model_diver_return_isna_long'] = \
                                    [(df_temp.diver_return[(df_temp.diver_return.isna()==False) & \
                                                                 (df_temp.sum_sentiment!=-1)]).sum()]






                                    res['train_accuracy'] = [metrics.accuracy_score(y_train_pred, y_train)]


                                    res['train_precision_macro'] = [metrics.precision_score(y_train_pred, y_train\
                                                                                            , average='macro')]


                                    res['train_precision_micro'] = [metrics.precision_score(y_train_pred, y_train\
                                                                                            , average='micro')]


                                    res['train_recall_micro'] = [metrics.recall_score(y_train_pred, y_train\
                                                                                      , average='micro')]


                                    res['train_recall_macro'] = [metrics.recall_score(y_train_pred, y_train\
                                                                                      , average='macro')]

                                    res['train_f1_micro'] = [metrics.f1_score(y_train_pred, y_train\
                                                                              , average='micro')]


                                    res['train_f1_macro'] = [metrics.f1_score(y_train_pred, y_train\
                                                                              , average='macro')]



                                    x = df_temp[['sum_sentiment', 'close_lag_1']]
                                    x.sum_sentiment = x.sum_sentiment.astype(str)

                                    x = pd.get_dummies(x, drop_first=True)
                                    y = df_temp.close
                                    x2 = sm.add_constant(x)
                                    lm = sm.OLS(y, x2)
                                    res_lm = lm.fit()

                                    res['const'] = res_lm.params[0]
                                    res[f'{x.columns[0]}_coef'] = res_lm.params[1]
                                    res[f'{x.columns[1]}_coef'] = res_lm.params[2]
                                    res[f'{x.columns[2]}_coef'] = res_lm.params[3]

                                    res['const_p'] = res_lm.pvalues[0]
                                    res[f'{x.columns[0]}_p'] = res_lm.pvalues[1]
                                    res[f'{x.columns[1]}_p'] = res_lm.pvalues[2]
                                    res[f'{x.columns[2]}_p'] = res_lm.pvalues[3]
                                    res['r2'] = res_lm.rsquared
                                    res['nobs'] = res_lm.nobs
                                    counter += 1
                                    print(counter, end='\r')


                                    res = pd.DataFrame.from_dict(res)

                                    out = out.append(pd.DataFrame(res))
                                    if counter % 100 == 0: 
                                        out.to_excel(f'/is/res/nb/mfd_nb_model_ind_final.xlsx', index=False)

                    except:
                        continue
    

                    


out.to_excel(f'/is/res/nb/mfd_nb_model_ind_final.xlsx', index=False)
out

tgka
sber
gazp
vtbr
mtss
mtlr
rsti
sngs
afks
yakg
chmf
irkt
gmkn
rolo
lsng
rosn
lkoh
nmtp
rual
tatn
mgnt
magn
yndx
alrs
selg
hydr
lsrg
lnta
qiwi
gche
akrn
fees
moex
upro
rtkm
plzl0
aflt8
dasb6
rasp4
gtlc2
